# A few calculations for FRB 171020 [v3]

In [1]:
# imports
import numpy as np

from astropy import units as u
from astropy.cosmology import Planck15
from astropy import constants

from ne2001 import density
from ne2001.density import NEobject

from pyigm.cgm.models import ModifiedNFW, MB04, YF17

## Coords

In [2]:
FRB_l = 36.4
FRB_b = -53.6

## $\rm DM_{\rm disk}$

### ne2001

In [3]:
ne = density.ElectronDensity()#**PARAMS)

In [4]:
DM_disk = ne.DM(FRB_l, FRB_b, 100.)
DM_disk

<Quantity 37.4206662443299 pc / cm3>

## $\rm DM_{\rm halo}$

### Halo properties

In [5]:
Mhalo = np.log10(1.5e12)
f_hot = 0.75  # Allows for disk + ISM
c = 7.7

### Simple Model

In [6]:
# MB
mb04 = MB04(log_Mhalo=Mhalo, f_hot=f_hot, c=c)
DM_halo = mb04.Ne_Rperp(8*u.kpc)  # This is approximate.  Should do proper l,b but may make little difference
DM_halo/2  # Only get 1/2 for our Galaxy

rhoV normalized to 0.00017228356281820233 1 / cm3 to give M_b=240000000000.00006 solMass


<Quantity 66.0997762729517 pc / cm3>

### Better model

In [7]:
params = dict(F=1., e_density=1.)
model_ne = NEobject(mb04.ne, **params)

In [8]:
DM_halo2 = model_ne.DM(FRB_l, FRB_b, mb04.r200.to('kpc').value)
DM_halo2

<Quantity 69.06644625604574 pc / cm3>

#### Inner 10kpc

In [9]:
DM_halo2_10 = model_ne.DM(FRB_l, FRB_b, 10.)
DM_halo2_10

<Quantity 7.93169677812895 pc / cm3>

## $\rm DM_{\rm host}$ -- This is Halo gas only

In [10]:
Mhost = np.log10(1e10)
f_hot_host = 0.5  # Allows for 50% blow-out
c = 7.7

In [11]:
mb04_host = MB04(log_Mhalo=Mhost, f_hot=f_hot_host, c=c)
DM_host = mb04_host.Ne_Rperp(1*u.kpc)
DM_host/2  # Only get 1/2

rhoV normalized to 7.048521943833164e-05 1 / cm3 to give M_b=1600000000.0 solMass


<Quantity 9.769291033825594 pc / cm3>

## D$_{\rm IGM}$

### Mean density

In [12]:
rhob = Planck15.Ob0 * Planck15.critical_density0
rhob

<Quantity 4.189050060781924e-31 g / cm3>

In [17]:
n_b = rhob / 1.3 / constants.m_p
n_b.decompose().cgs

<Quantity 1.926523982768926e-07 1 / cm3>

### Scale down by baryons in stars, compact objects

In [23]:
neff_b = n_b * 0.7 # Approximate

### Do it

In [19]:
DM_IGM = 50 * u.pc / u.cm**3

In [24]:
D_IGM = DM_IGM / neff_b
D_IGM.to('Mpc')

<Quantity 370.7639877179709 Mpc>